In [1]:
import sys

import numpy as np

import torch
from torch.utils.data import DataLoader

from transformers import DistilBertTokenizer

import pickle as pkl

sys.path.insert(0, '..')

from src.data_collection import get_data
from src.models import (
    HateDataset, DistilBERTMultiClass, get_distil_hyperparams, predict_distilbert,
    predict_bert_tweet_roberta, prepare_dataloaders, get_bertweet_hyperparams,
    get_roberta_hyperparams
)

In [2]:
hate_speech_ucb = get_data()

device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"

N_SAMPLES = 1000

hate_speech_sample = hate_speech_ucb[:N_SAMPLES]#.sample(N_SAMPLES, random_state=585)
N_CLASSES = hate_speech_sample["hatespeech"].nunique()

Fetching data...


Using custom data configuration ucberkeley-dlab--measuring-hate-speech-7cb9b0b8e4d0e1dd
Reusing dataset parquet (C:\Users\UTKARSH\.cache\huggingface\datasets\parquet\ucberkeley-dlab--measuring-hate-speech-7cb9b0b8e4d0e1dd\0.0.0\0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/1 [00:00<?, ?it/s]

Processing...
Done!


In [3]:
BERTWEET_HYPERPARAMS = get_bertweet_hyperparams()

bertweet_model_path = BERTWEET_HYPERPARAMS["MODEL_PATH"]

with open(bertweet_model_path, "rb") as f:
    bertweet_large_model = pkl.load(f)

BERTWEET_MODEL = BERTWEET_HYPERPARAMS["MODEL_NAME"]
batch_size = BERTWEET_HYPERPARAMS["BATCH_SIZE"]

bertweet_roberta_dataloader = prepare_dataloaders(hate_speech_sample, BERTWEET_MODEL, batch_size)
bertweet_outputs = predict_bert_tweet_roberta(bertweet_large_model, bertweet_roberta_dataloader, device)
bertweet_outputs = np.argmax(bertweet_outputs, axis = 1)

In [4]:
DISTIL_HYPERPARAMS = get_distil_hyperparams()

distil_model_path = DISTIL_HYPERPARAMS["MODEL_PATH"]
distil_vocab_path = DISTIL_HYPERPARAMS["VOCAB_PATH"]

DISTIL_TOKENIZER = DistilBertTokenizer.from_pretrained(distil_vocab_path)
DISTIL_MAX_LEN = DISTIL_HYPERPARAMS["MAX_LEN"]

distil_params = DISTIL_HYPERPARAMS["TEST_PARAMS"]
distil_dataset = HateDataset(hate_speech_sample, DISTIL_TOKENIZER, DISTIL_MAX_LEN)
distil_dataloader = DataLoader(distil_dataset, **distil_params)

distil_model = DistilBERTMultiClass(n_classes=N_CLASSES)
distil_model.load_state_dict(torch.load(distil_model_path))

distil_outputs = predict_distilbert(distil_model, distil_dataloader, device)
distil_outputs = np.argmax(distil_outputs, axis=1)

C:\Users\UTKARSH\miniconda3\envs\env_deep\lib\site-packages\transformers\tokenization_utils_base.py:1653: FutureWarning: Calling DistilBertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCla

In [5]:
ROBERTA_HYPERPARAMS = get_roberta_hyperparams()

roberta_model_path = ROBERTA_HYPERPARAMS["MODEL_PATH"]

with open(roberta_model_path, "rb") as f:
    roberta_model = pkl.load(f)
roberta_outputs = predict_bert_tweet_roberta(roberta_model, bertweet_roberta_dataloader, device)
roberta_outputs = np.argmax(roberta_outputs, axis=1)

In [6]:
hate_speech_sample["RoBERTa_pred"] = roberta_outputs
hate_speech_sample["DistilBERT_pred"] = distil_outputs
hate_speech_sample["BERTweet_Large_pred"] = bertweet_outputs

C:\Users\UTKARSH\AppData\Local\Temp\ipykernel_12424\2550004462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_speech_sample["RoBERTa_pred"] = roberta_outputs
C:\Users\UTKARSH\AppData\Local\Temp\ipykernel_12424\2550004462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_speech_sample["DistilBERT_pred"] = distil_outputs
C:\Users\UTKARSH\AppData\Local\Temp\ipykernel_12424\2550004462.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

### Errors - RoBERTa

In [7]:
roberta_errors = hate_speech_sample[
    hate_speech_sample["hatespeech"] != hate_speech_sample["RoBERTa_pred"]
]

### Errors - RoBERTa and DistilBERT combined

In [8]:
roberta_distil_errors = roberta_errors[
    roberta_errors["hatespeech"] != roberta_errors["DistilBERT_pred"]
]

### Errors - RoBERTa, DistilBERT and BERTweet combined

In [9]:
roberta_distil_bertweet_errors = roberta_distil_errors[
    roberta_distil_errors["hatespeech"] != roberta_distil_errors["BERTweet_Large_pred"]
]

In [10]:
text_list = roberta_distil_bertweet_errors["text"].tolist()
labels_list = roberta_distil_bertweet_errors["hatespeech"].tolist()

#### Hate Speech without swear words

In [11]:
for text, label in zip(text_list, labels_list):
    if label == 1:
        print("(Hate Speech without swear words)\t", text)
        print("-" * 150)

(Hate Speech without swear words)	 "Bang outside, I nutted on god" ok 21 go off?
------------------------------------------------------------------------------------------------------------------------------------------------------
(Hate Speech without swear words)	 "He means well, but Trump just hasn't been in Washington long enough to understand its inner workings the way we do. When it comes to advancing white power, you can't let the perfect be the enemy of the good."
------------------------------------------------------------------------------------------------------------------------------------------------------
(Hate Speech without swear words)	 "I had fuck more than you, oh look at me, poor thing, getting a dick inside me! Please let me cut in line!"
------------------------------------------------------------------------------------------------------------------------------------------------------
(Hate Speech without swear words)	 "I hope you get raped and more" well shit
-

#### Non Hate Speech with swear words

In [12]:
for text, label in zip(text_list, labels_list):
    if label == 0:
        print("(Non Hate Speech with swear words)\t", text)
        print("-" * 150)

(Non Hate Speech with swear words)	 "Eating pussy - God thats so gay" Every retard ever.
------------------------------------------------------------------------------------------------------------------------------------------------------
(Non Hate Speech with swear words)	 "Fuck off" yeah man, fuck those white children being butchered. Yes, they're there from colonialism- aren't you smart? But they still don't deserve that. Grow the fuck up and stop being a race-hating cuck.
------------------------------------------------------------------------------------------------------------------------------------------------------
(Non Hate Speech with swear words)	 "Mom, what's a cuck?" I can't believe you sheep followed this pussy fart over the cliff. All of you spineless cucks enjoy walking your wife's boyfriend's dog.
------------------------------------------------------------------------------------------------------------------------------------------------------
(Non Hate Speech with

### Error Analysis

We see a few sentences which clearly show that being hate-speech does not necessarily depend on whether the sentence contains swear words or not.

Here are a few examples that __do not have__ swear words but are still __Hate speech__:

- "He means well, but Trump just hasn't been in Washington long enough to understand its inner workings the way we do. When it comes to advancing white power, you can't let the perfect be the enemy of the good."
- "If you consider yourself a feminist, but your thoughts on trans women are virtually identical to the raging misogynists and ignorant anti-feminists who make up the incel movement, well, maybe you're not as much of a feminist as you think. " URL
- "Iranians have nice and cozy graves to sleep in during the winter, it's strategically vital to make sure Hamas has a hearty Eftar, that will show our enemies."

---

Now, here we have a few examples which __do have__ swear words but are still __Non-hate speech__:

- "Fuck off" yeah man, fuck those white children being butchered. Yes, they're there from colonialism- aren't you smart? But they still don't deserve that. Grow the fuck up and stop being a race-hating cuck.
- "lol everyone get a look at this faggot"
- "Mom, what's a cuck?" I can't believe you sheep followed this pussy fart over the cliff. All of you spineless cucks enjoy walking your wife's boyfriend's dog.

---

These are the sentences where our models did not perform well. As we can see, these sentences are difficult to identify, so for future improvements, we need to consider this to train better models capable of identifying hate speech, even from texts that do not look that bad but can still offend someone.